In [71]:
import urllib2, json
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [36]:
# Get all csv file pathes from the source directory
path = "CSV_merged/"
files = [join(path, f) for f in listdir(path) if isfile(join(mypath, f))]

In [59]:
# Merge the istex_id(s) and export the unique ones in a merged CSV file
merged = np.array([], dtype=np.object)
for f in files:
    t = pd.read_csv(f,header=None, names=['istex_id'])
    t = np.array(t['istex_id'])
    merged = np.hstack((merged,t))

unique_merged = np.unique(merged)
print "number of merged documents:", len(merged)
print "number of unique documents:", len(unique_merged)


number of merged documents: 341728
number of unique documents: 292150


In [63]:
# Export the unique merged list of istex_ids into a CSV file
istex_ids = pd.DataFrame(unique_merged)
istex_ids.to_csv("unique_merged.csv", index=None, header=None)

In [64]:
list_themes = ["Artificial Intelligence","Information Systems", "Rehabilitation", "Philosophy","Microscopy","Infectious Diseases","Respiratory System", \
"Literature","Robotics", "Pediatrics", "Mechanics", "Condensed Matter", "Transplantation", "Religion", "Pathology", "Immunology", "Nursing", "Substance Abuse", "Thermodynamics", "Psychology", 
"Ophthalmology", "Ceramics", "Toxicology", "Neuroimaging", "Sociology", "Psychiatry", "Oncology", "biophysics", "Emergency Medicine", "Surgery", "Physiology", "Mycology", "Biomaterials"]
len(list_themes)

33

In [90]:
def query_from(q, f):
	q = q+'&from='+str(f)
	response = urllib2.urlopen(q)
	data = json.load(response)
	subject_ids = np.array(range(len(data['hits'])), dtype=np.object)
	for (i, hit) in enumerate(data['hits']):
		subject_ids[i] = hit['id']
	return subject_ids

def query(q):
	response = urllib2.urlopen(q)
	data = json.load(response)
	nb_requests = 1 + data['total'] / 1000
	if nb_requests > 10: # maximum number of pages due to API pagination restrection
		nb_requests = 10
	subject_ids = query_from(q, 0)
	for i in range(nb_requests)[1:]:
		f = i * 1000
		next_request = query_from(q, f)
		subject_ids = np.hstack((subject_ids, next_request))
	return subject_ids.tolist()

def term2url(string):
	string = string.split(' ')
	res = '%22'
	for s in string:
		res = res + s + '%20'
	res = res[:-3]
	res = res + '%22'
	return res

def keyword(topic):
	q = 'https://api.istex.fr/document/?q=(('
	topic = term2url(topic)
	q = q+ 'keywords.teeft:' + topic
	q = q + ')%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20ANvdD%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'
	return q

def title_abst(topic):
	q = 'https://api.istex.fr/document/?q=(('
	topic = term2url(topic)
	q = q+ 'title:' + topic + '%20OR%20abstract:' + topic
	q = q + ')%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20ANvdD%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'
	return q


In [93]:
q = title_abst(list_themes[0])
print q
res = query(q)
res[:10]

https://api.istex.fr/document/?q=((title:%22Artificial%20Intelligence%22%20OR%20abstract:%22Artificial%20Intelligence%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20ANvdD%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id


[u'166C3E42883D652A04179CB5ADE02F07F8482B24',
 u'099ED2EA72F069B6E17185C0C2E21FF3A08B71BA',
 u'D87830C0D057EFC73C14896C18A57EAFA5C09628',
 u'C5D233E6AB2808BEA83BBA252D1B9872D7B9E14D',
 u'974F694F33A407D81A2FE0EC1455039E4AC42B63',
 u'B3D478E9FABF61920E760388085E6D931BD40A82',
 u'A1AFF75E4F64C29DA9C45508024BB440D0840FAF',
 u'608FF2B1E3C1343B89F4DAC98C6B36B62BBE1F76',
 u'4A74948C95085026897F01BE4F01A690535A191A',
 u'D3181511C70A95DBDC475D9D998D6C6A1846B33B']

In [94]:
q = keyword(list_themes[0])
print q
res = query(q)
res[:10]

https://api.istex.fr/document/?q=((keywords.teeft:%22Artificial%20Intelligence%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20ANvdD%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id


[u'AEC340AA0D3F7BC6C1A3F1ABB4438D211FC74F4A',
 u'F2E0B16D2A22150FC5007051AB16510C81237370',
 u'05E4E6A76E75CE9820F59D5516A4F3BBCB8A6EE6',
 u'7DD3736FAF2D2B7F02FCE4A423C059EB97BFAC61',
 u'344CF39480130E1A468DB495AFE0A70D83CC057D',
 u'57BF6B99AAE986CA760B33CF830C797EF91BE9C3',
 u'D496043AC37B0A72C22A648BC3805EA9ADD14FFE',
 u'29261003D672AAC7ACE6762CDCD45EA51C756D5C',
 u'93F822A46600067B50B2162453492A5127628E16',
 u'EB553E80949D3D095C47F76C7C677EA4E095C83B']